In [1]:
import time
start = time.time()

import gurobipy as gb
import numpy as np
import csv

In [2]:
# Class which can have attributes set.
class expando(object):
    pass

In [3]:
# Master problem
class Relaxed_Master:
    def __init__(self,count,gam,ep=0.000001, de=0.0001,scenarios=200):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.params = expando()
        self.params.scenarios = scenarios
        self.params.gam = gam
        self.params.i = count
        self._load_data()
        self._build_model()
        self._init_benders_params(ep=ep, de=de)
        self._start_from_previous()
    
    def optimize(self):
        self.model.Params.lazyConstraints = 1
        #self.model.Params.Threads = 1
        start1 = time.time()
        
        self.subs = {}
        self.submodels = {}
        self.feas_model = {}
        self.parameter_problems = {}
        #self.opt_parameter_problems = {}
        for o in range(self.data.O):
            self.submodels[o] = {}
            self.submodels[o][0] = Subproblem_ISSPo(self,o)
            self.submodels[o][1] = Subproblem_NISSPo(self,o)
            self.parameter_problems[o] = parameter_problemo(self,o)
            #self.opt_parameter_problems[o] = opt_parameter_problemo(self,o)
        end1 = time.time()    
        print("Time spent to pre-process the code: " + str(end1-start1), file=open("output16-800-"+str(self.params.i+1)+".txt", "a"))    
        self.model._vars = self.model.getVars()
        #self.model.Params.TimeLimit = 1000
        self.model.Params.LogToConsole = 0
        self.model.Params.LogFile = "sol16-800-"+str(self.params.i+1)
        start = time.time()
        #self.model.optimize(callbackfunc)
        end = time.time()
        print ("Time taken by the code for the instance " + str(self.params.i+1) + "= " + str(end-start), file=open("output16-800-"+str(self.params.i+1)+".txt", "a"))
        print ("Time taken for the Kappa " + str(self.params.i+1) + "= " + str(self.data.extra1), file=open("output16-800-"+str(self.params.i+1)+".txt", "a"))
        print ("Time taken by the extra linearization in the code " + "= " + str(self.data.extra), file=open("output16-800-"+str(self.params.i+1)+".txt", "a"))  
        
        #print ("Upper Bound: "+str(self.data.ub), file=open("output16-800-"+str(self.params.i+1)+".txt", "a"))  
        print ("Lower Bound: "+str(self.data.lb), file=open("output16-800-"+str(self.params.i+1)+".txt", "a"))
        #print (str(self.data.lb), file=open("output16-800-"+str(1)+".txt", "a"))
        #print ("MIP Gap:" + str((((m.data.ub-m.data.lb)*100)/m.data.lb)), file=open("output16-800-"+str(self.params.i+1)+".txt", "a"))

            
    def _init_benders_params(self, ep=0.000001, de=0.000001):
        self.data.upper_bounds = []
        self.data.lower_bounds = []
        self.data.ub = gb.GRB.INFINITY
        self.data.lb = -gb.GRB.INFINITY 

    
    def _load_data(self):
        self.data.I = 30
        self.data.J = 40
        self.data.O = self.params.scenarios
        self.data.extra = 0
        self.data.extra1 = 0
        self.data.X = np.zeros((1,self.data.I))
        self.data.p_x = np.zeros(self.data.O) 
        self.data.p_o = 1/self.data.O
        self.data.gama = self.params.gam
        print ("Gamma: "+str(self.data.gama), file=open("output16-800-"+str(self.params.i+1)+".txt", "a"))
        #fil_name = 'demand-'+str(self.params.i+1)
        fil_name = 'demand-'+str(200)+'-'+str(self.params.i+1)
        with open(fil_name+'.csv', 'r') as f:
            reader = csv.reader(f)
            examples = list(reader)
        self.data.demand = []
        for row in examples:
            nwrow = []
            for r in row:
                nwrow.append(eval(r))
            self.data.demand.append(nwrow)
        """     
        fil_name = 'X-100'
        with open(fil_name+'.csv', 'r') as f:
            reader = csv.reader(f)
            examples = list(reader)
        self.data.X1 = []
        for row in examples:
            nwrow = []
            for r in row:
                nwrow.append(eval(r))
            self.data.X1.append(nwrow)          
            
           
        fil_name = 'A-'+str(self.params.i+1)
        with open(fil_name+'.csv', 'r') as f:
            reader = csv.reader(f)
            examples = list(reader)
        self.data.A = []
        for row in examples:
            nwrow = []
            for r in row:
                nwrow.append(eval(r))
            self.data.A.append(nwrow)
            
        fil_name = 'B-'+str(self.params.i+1)
        with open(fil_name+'.csv', 'r') as f:
            reader = csv.reader(f)
            examples = list(reader)
        self.data.B = []
        for row in examples:
            nwrow = []
            for r in row:
                nwrow.append(eval(r))
            self.data.B.append(nwrow)            
        """
        #self.data.Cx = [0.98, 1.01, 0.91, 0.74, 0.74, 1.29, 1.05, 0.88, 0.88, 1.15]
        #self.data.Cx = [0.89,  0.95,  0.82,  1.13,  0.76,  0.99,  1.04,  1.05,  1.15,  0.9, 0.79,  0.91,  0.92,  0.96,  1.03,  0.93,  0.99,  1.08,  1.03,  0.99]
        self.data.Cx = [0.78524047, 1.03853684, 1.08658277, 1.01837842, 1.28717483,
                           0.95977712, 0.63889886, 1.02319385, 0.91436706, 1.09957874,
                           1.44681779, 0.84262975, 0.86435203, 0.87074037, 1.60196848,
                           1.00862552, 1.44129719, 0.98423685, 1.21601241, 0.82694264,
                           1.00078404, 1.08870653, 0.86454873, 0.58476141, 1.07898004,
                           1.20424872, 1.1846476 , 0.79578763, 0.91078195, 1.24328623]
        self.data.Cy = [2.3130311834976878, 1.970110451642846, 2.079834948369165, 2.180997535033276, 2.1887583154880983, 2.157134900936415, 2.148981366909944, 2.063591377530072, 2.114553485889044, 2.2016770965819905, 1.9966732632984514, 2.2330830064352423, 2.073101936010451, 2.078736481418292, 2.1023788717803735, 2.20322688021812, 2.0598950590963474, 2.186612946072563, 2.2213945559591637, 2.4194867761920977, 2.177263511555545, 2.2031710809006455, 1.9909192466456946, 2.2396747891757802, 2.1661606250152716, 2.049535136705102, 2.1171635559467483, 2.2147607076134714, 2.203825916684725, 2.3108454507501515]
        self.data.rho = [0.9555664467976429, 0.9739615542635913, 0.9122667177690378, 0.9340610389388502, 0.9246641057347171, 0.9945151890629177, 0.9615143263620417, 0.9659480369131807, 0.9358045688085963, 0.9147377425339717, 0.9756453545272545, 0.9350544644953063, 0.9839239868461557, 0.9953649754312432, 0.9288115703035881, 0.9946050201412094, 0.9575793792880826, 0.9224016204902284, 0.905702317419421, 0.9507354965994901]
        
        fil_name = 'mu-30-40'
        with open(fil_name+'.csv', 'r') as f:
            reader = csv.reader(f)
            examples = list(reader)
        self.data.mu = []
        for row in examples:
            nwrow = []
            for r in row:
                nwrow.append(eval(r))
            self.data.mu.append(nwrow)
        
        self.data.epsilon = 0.1
        
        
        self.data.p = np.zeros(self.data.O)
        self.data.probs = []
        for o in range(self.data.O):
            self.data.p[o] = self.data.p_o
        p = [self.data.p]
        self.data.probs += p
        
        sm = parametero(self)
        sm.model.setParam("OutputFlag", 0 )  
        
        start2 = time.time()
        sm.model.optimize() 
        end2 = time.time()
        self.data.Kappa = sm.model.ObjVal
        
        self.data.extra1 += (end2-start2)
        
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()  
        
    def _build_variables(self):
        m = self.model
        self.variables.X = m.addVars([(a) for a in range(self.data.I)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="X_snt")
        self.variables.beta = m.addVars([(a) for a in range(self.data.O)], vtype=gb.GRB.BINARY, name="beta_o")
        m.update()
        
    def _build_objective(self):
        obj = sum(self.variables.X[(i)]*self.data.Cx[i] for i in range(self.data.I))
        
        self.model.setObjective(obj, gb.GRB.MINIMIZE)
        
    def _build_constraints(self):
        m = self.model
        self.constraints.c1 = m.addConstr((sum(self.data.p_o*self.variables.beta[o] for o in range(self.data.O)) <= self.data.epsilon)) 
        #self.constraints.c2 = m.addConstr(((0.5)*sum(self.variables.t[i] for i in range(self.data.O)) <= self.data.gama), "c5")
        #self.constraints.c2 = m.addConstrs((self.variables.p_x[i] - self.variables.t[i] <= self.data.p_o for i in range(self.data.O)), "c6")
        #self.constraints.c3 = m.addConstrs((- self.variables.p_x[i] - self.variables.t[i] <= - self.data.p_o for i in range(self.data.O)), "c7")
        #self.constraints.c5 = m.addConstr((sum(self.variables.p_x[(i)] for i in range(self.data.O)) == 1), "c8")
        pass
        
    def _start_from_previous(self):
        '''
            Used to warm-start MIP problems.
        '''
        pass


In [4]:
class opt_parameter_problemo:
    def __init__(self, MP, scenario):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.results = expando()
        self.MP = MP
        self.data.scenario = scenario
        self._build_model()
      

    def optimize(self):
        self.model.optimize()
        
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()
        
    def _build_variables(self):
        m = self.model
        self.variables.X = m.addVars([(b) for b in range(self.MP.data.I)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_X_osrt")
        self.variables.Y = m.addVars([(b, c) for b in range(self.MP.data.I) for c in range(self.MP.data.J)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_X_osrt")
        self.variables.phi = m.addVars([(b) for b in range(self.MP.data.J)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_X_osrt")
        m.update()
        
    def _build_objective(self):
        m = self.model
        obj = 0
        self.model.setObjective(obj, gb.GRB.MINIMIZE)
        
    def _build_constraints(self):
        m = self.model
        self.constraints.c1 = m.addConstrs((sum(self.variables.Y[(i,j)] for j in range(self.MP.data.J)) <= self.variables.X[(i)] for i in range(self.MP.data.I)), "c1")
        self.constraints.c2 = m.addConstrs((sum(self.MP.data.mu[i][j]*self.variables.Y[(i,j)] for i in range(self.MP.data.I)) + self.variables.phi[(j)] >= self.MP.data.demand[self.data.scenario][j] for j in range(self.MP.data.J)), "c4")
        

In [5]:
class parameter_problemo:
    def __init__(self, MP, scenario):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.results = expando()
        self.MP = MP
        self.data.scenario = scenario
        self._build_model()
      

    def optimize(self):
        self.model.optimize()
        
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()
        
    def _build_variables(self):
        m = self.model
        self.variables.X = m.addVars([(b) for b in range(self.MP.data.I)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_X_osrt")
        self.variables.Y = m.addVars([(b, c) for b in range(self.MP.data.I) for c in range(self.MP.data.J)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_X_osrt")
        m.update()
        
    def _build_objective(self):
        m = self.model
        obj = 0
        self.model.setObjective(obj, gb.GRB.MINIMIZE)
        
    def _build_constraints(self):
        m = self.model
        self.constraints.c1 = m.addConstrs((sum(self.variables.Y[(i,j)] for j in range(self.MP.data.J)) <= self.variables.X[i] for i in range(self.MP.data.I)), "c1")
        self.constraints.c2 = m.addConstrs((sum(self.MP.data.mu[i][j]*self.variables.Y[(i,j)] for i in range(self.MP.data.I)) >= self.MP.data.demand[self.data.scenario][j] for j in range(self.MP.data.J)), "c4")
                

In [6]:
class Subproblem_ISSPo:
    def __init__(self, MP, scenario):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.results = expando()
        self.MP = MP
        self.data.scenario = scenario
        self._build_model()
      

    def optimize(self):
        self.model.optimize()
        
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()
        
    def _build_variables(self):
        m = self.model
        self.variables.Y = m.addVars([(b, c) for b in range(self.MP.data.I) for c in range(self.MP.data.J)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_X_osrt")
        m.update()
        
    def _build_objective(self):
        m = self.model
        obj = 0
        self.model.setObjective(obj, gb.GRB.MINIMIZE)
        
    def _build_constraints(self):
        m = self.model
        self.constraints.c1 = m.addConstrs((sum(self.variables.Y[(i,j)] for j in range(self.MP.data.J)) <= 0 for i in range(self.MP.data.I)), "c1")
        self.constraints.c2 = m.addConstrs((sum(self.MP.data.mu[i][j]*self.variables.Y[(i,j)] for i in range(self.MP.data.I)) >= self.MP.data.demand[self.data.scenario][j] for j in range(self.MP.data.J)), "c4")
        

In [7]:
class Subproblem_NISSPo:
    def __init__(self, MP, scenario):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.results = expando()
        self.MP = MP
        self.data.scenario = scenario
        self._build_model()
      

    def optimize(self):
        self.model.optimize()
        
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()
        
    def _build_variables(self):
        m = self.model
        self.variables.Y = m.addVars([(b, c) for b in range(self.MP.data.I) for c in range(self.MP.data.J)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_X_osrt")
        self.variables.phi = m.addVars([(b) for b in range(self.MP.data.J)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_X_osrt")
        m.update()
        
    def _build_objective(self):
        m = self.model
        obj = 0
        self.model.setObjective(obj, gb.GRB.MINIMIZE)
        
    def _build_constraints(self):
        m = self.model
        self.constraints.c1 = m.addConstrs((sum(self.variables.Y[(i,j)] for j in range(self.MP.data.J)) <= 0 for i in range(self.MP.data.I)), "c1")
        self.constraints.c2 = m.addConstrs((sum(self.MP.data.mu[i][j]*self.variables.Y[(i,j)] for i in range(self.MP.data.I)) + self.variables.phi[(j)] >= self.MP.data.demand[self.data.scenario][j] for j in range(self.MP.data.J)), "c4")
        

In [8]:
class parametero:
    def __init__(self, MP):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.results = expando()
        self.MP = MP
        self._build_model()
      

    def optimize(self):
        self.model.optimize()
        
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()
        
    def _build_variables(self):
        m = self.model
        self.variables.Z = m.addVars([(b) for b in range(self.MP.data.O)], lb = 0, vtype=gb.GRB.BINARY, name="Z")
        self.variables.P = m.addVars([(b) for b in range(self.MP.data.O)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="P")
        self.variables.t = m.addVars([(b) for b in range(self.MP.data.O)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="pi")
        self.variables.pi0 = m.addVar(lb = 0, vtype=gb.GRB.CONTINUOUS, name="pi")
        self.variables.pi1 = m.addVars([(b) for b in range(self.MP.data.O)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="pi")
        self.variables.pi2 = m.addVars([(b) for b in range(self.MP.data.O)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="pi")
        self.variables.pi3 = m.addVar(vtype=gb.GRB.CONTINUOUS, name="pi")
        #self.variables.pi32 = m.addVar(vtype=gb.GRB.CONTINUOUS, lb = 0, name="pi")
        m.update()
        
    def _build_objective(self):
        m = self.model
        obj = sum(self.variables.Z[i] for i in range(self.MP.data.O))
        self.model.setObjective(obj, gb.GRB.MAXIMIZE)
        
    def _build_constraints(self):
        m = self.model
        self.constraints.c1 = m.addConstr((sum((self.variables.pi1[i]-self.variables.pi2[i])*self.MP.data.p_o for i in range(self.MP.data.O)) + self.variables.pi3 + self.MP.data.gama*self.variables.pi0 <= self.MP.data.epsilon), "c1")
        self.constraints.c2 = m.addConstrs((0.5*self.variables.pi0 - self.variables.pi1[i] - self.variables.pi2[i] >= 0 for i in range(self.MP.data.O)), "c2")
        self.constraints.c3 = m.addConstrs((self.variables.pi1[i] - self.variables.pi2[i] + self.variables.pi3 >= self.variables.Z[i] for i in range(self.MP.data.O)), "c3")
        self.constraints.c4 = m.addConstr((sum((self.variables.pi1[(i)]-self.variables.pi2[(i)])*self.MP.data.p_o for i in range(self.MP.data.O))  + self.variables.pi3 + self.MP.data.gama*self.variables.pi0 == sum(self.variables.P[i]*self.variables.Z[i] for i in range(self.MP.data.O))), "c4")
        self.constraints.c5 = m.addConstr(((0.5)*sum(self.variables.t[i] for i in range(self.MP.data.O)) <= self.MP.data.gama), "c5")
        self.constraints.c6 = m.addConstrs((self.variables.P[i] - self.variables.t[i] <= self.MP.data.p_o for i in range(self.MP.data.O)), "c6")
        self.constraints.c7 = m.addConstrs((- self.variables.P[i] - self.variables.t[i] <= - self.MP.data.p_o for i in range(self.MP.data.O)), "c7")
        #self.constraints.c8 = m.addConstr((sum(self.variables.P[(i)]*self.variables.Z[(i)] for i in range(self.MP.data.O)) <= self.MP.data.epsilon), "c8")
        self.constraints.c9 = m.addConstr((sum(self.variables.P[i] for i in range(self.MP.data.O)) == 1), "c8")
        

In [9]:
class parametero1:
    def __init__(self, MP, beta):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.results = expando()
        self.MP = MP
        self.beta = beta
        self._build_model()
      

    def optimize(self):
        self.model.optimize()
        
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()
        
    def _build_variables(self):
        m = self.model
        #self.variables.Z = m.addVars([(b) for b in range(self.MP.data.O)], lb = 0, vtype=gb.GRB.BINARY, name="Z")
        self.variables.P = m.addVars([(b) for b in range(self.MP.data.O)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="P")
        self.variables.t = m.addVars([(b) for b in range(self.MP.data.O)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="pi")
        
        m.update()
        
    def _build_objective(self):
        m = self.model
        obj = sum(self.beta[i]*self.variables.P[(i)] for i in range(self.MP.data.O))
        self.model.setObjective(obj, gb.GRB.MAXIMIZE)
        
    def _build_constraints(self):
        m = self.model
        self.constraints.c1 = m.addConstr(((0.5)*sum(self.variables.t[(i)] for i in range(self.MP.data.O)) <= self.MP.data.gama), "c5")
        self.constraints.c2 = m.addConstrs((self.variables.P[i] - self.variables.t[(i)] <= self.MP.data.p_o for i in range(self.MP.data.O)), "c6")
        self.constraints.c3 = m.addConstrs((- self.variables.P[i] - self.variables.t[(i)] <= - self.MP.data.p_o for i in range(self.MP.data.O)), "c7")
        self.constraints.c4 = m.addConstr((sum(self.variables.P[(i)] for i in range(self.MP.data.O)) == 1), "c8")

In [10]:
def callbackfunc(model, where):      
    
    if where == gb.GRB.Callback.MIPSOL:
        Feas_flag = False
        Cut_found = False
        vList = model.cbGetSolution(model.getVars())
        
        
        z = model.cbGet(gb.GRB.Callback.MIPSOL_OBJ)
        
        
        X = np.zeros(m.data.I)
        
        beta_o = np.zeros(m.data.O)
        
        for o in range(m.data.O):
            
            beta_o[o] = vList[m.data.I + o]
        for i in range(m.data.I):
            X[i] = vList[i]
        
        sub = np.zeros(m.data.O) 
         
        for scenario in range(m.params.scenarios):
            
            if (Feas_flag == True):
                break
                
            if (beta_o[scenario] == 0):
                start2 = time.time()
                sm = m.submodels[scenario][0]
                sm.model.setParam ( "OutputFlag", 0 )
                for i in range(m.data.I):
                    sm.constraints.c1[(i)].rhs = X[i] 
                obj = 0
                sm.model.setObjective(obj, gb.GRB.MINIMIZE)     
                end2 = time.time()
                m.data.extra += (end2-start2)
                sm.model.optimize()
                    
                    
                if (sm.model.status == gb.GRB.INFEASIBLE):
                    #print ("Here", file=open("output16-800-"+str(m.params.i+1)+".txt", "a"))
                    start2 = time.time()
                    sm.model.setObjective(obj, gb.GRB.MINIMIZE) 
                    sm.model.setParam ( "OutputFlag", 0 )
                    sm.model.setParam ( "InfUnbdInfo", 1 )
                    end2 = time.time()
                    m.data.extra += (end2-start2)
                    sm.model.optimize()
                    
                    lambda1 = np.zeros(m.data.I)
                      
                    for i in range(m.data.I):
                        lambda1[i] = sm.constraints.c1[(i)].FarkasDual
                                
                    h_val = {}    
                    
                    for o in range(m.data.O):
                        start2 = time.time()
                        sm1 = m.parameter_problems[o]
                        obj = sum(lambda1[i]*sm1.variables.X[(i)] for i in range(m.data.I))
                        sm1.model.setObjective(obj, gb.GRB.MINIMIZE)
                        sm1.model.setParam ("OutputFlag", 0) 
                        end2 = time.time()
                        m.data.extra += (end2-start2)
                        sm1.model.optimize()
                        h_val[o] = sm1.model.ObjVal
                    h_zeta = {k: v for k, v in sorted(h_val.items(), key=lambda item: item[1], reverse=True)}
                    m.data.keys = list(h_zeta.keys())
                    
                    if (beta_o[m.data.keys[0]] == 0):
                        m.model.cbLazy(sum(model._vars[i]*lambda1[(i)] for i in range(m.data.I)) +
                                    (h_val[int(m.data.keys[0])] - h_val[int(m.data.keys[int(m.data.Kappa)])])*model._vars[m.data.I+ int(m.data.keys[0])] >= h_val[int(m.data.keys[0])])
                        
                        
                    else: 
                        s = 0
                        for i in range(int(m.data.Kappa)+1):
                            if (beta_o[int(m.data.keys[i+1])] == 0):
                                s = i+1
                                break
                                
                        m.model.cbLazy(sum(model._vars[i]*lambda1[(i)] for i in range(m.data.I)) +
                                                (h_val[int(m.data.keys[0])] - h_val[int(m.data.keys[s])])*(model._vars[m.data.I+ int(m.data.keys[0])]) +
                                                (h_val[int(m.data.keys[s])] - h_val[int(m.data.keys[int(m.data.Kappa)])])*(model._vars[m.data.I + int(m.data.keys[s])]) >= h_val[int(m.data.keys[0])])
                    #end2 = time.time()
                    #m.data.extra += (end2-start2)
                        
                            
                        
                    Feas_flag = True

                    
                
        if (True):
            for i in range(m.data.I):
                m.data.X[0][i] = X[i]
            start2 = time.time()
            sm = parametero1(m,beta_o)
            sm.model.setParam ( "OutputFlag", 0 )    
            end2 = time.time()
            m.data.extra += (end2-start2)
            sm.model.optimize()
            if(sm.model.ObjVal <= m.data.epsilon):
                True
            else:
                p = np.zeros(m.data.O)
                for i in range(m.data.O):
                    p[i] = sm.variables.P[i].x
                prob = [p]
                m.data.probs += prob
                #start2 = time.time()
                m.model.cbLazy((sum(p[o]*m.variables.beta[o] for o in range(m.data.O)) <= m.data.epsilon))
                #end2 = time.time()
                #m.data.extra += (end2-start2)

            m.data.lb = z
            
            
    

In [11]:
for j in range(5):
    for i in range(7):
        gam = (i)*0.02
        m = Relaxed_Master(j, gam)
        m.optimize()
        """
        fil_name = 'X-600'
        example = m.data.X
        example = example.tolist()
        with open(fil_name+'.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerows(example)
        """

Using license file /software/commercial/gurobi/gurobi.lic
Set parameter TokenServer to value license8.clemson.edu
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of